In [33]:
import keras
import h5py
import numpy as np
import random
import cv2
from keras_tqdm import TQDMNotebookCallback
from tqdm import tqdm_notebook

infodir = 'C:/Science_Research/xraynet_data/images'
test_path = f'{infodir}/test.h5'
train_path = f'{infodir}/train.h5'
MODEL_NAME ='xrayNet-keras-CapsNet-v1' #gve the model a name
IMG_SIZE = 128
# input image dimensions

In [3]:
from matplotlib import pyplot as plt
import h5py

h5f = h5py.File(test_path, 'r')
image = h5f['X'][11]
print(h5f['Y'][11])
plt.imshow(image, cmap='gray')
plt.show()
im = image*255
im = cv2.equalizeHist(im.astype('uint8'))
plt.imshow(im.astype('float32'), cmap='gray')
plt.show()
h5f.close()

[1. 0.]


In [4]:
def load_val():
    h5f = h5py.File(test_path, 'r')
    X_val_pre = h5f['X']
    Y_val_pre = h5f['Y']

    X_val = []
    Y_val = []



    for image in tqdm_notebook(X_val_pre):
        im = image*255
        eq = cv2.equalizeHist(im.astype('uint8'))
        im = eq.astype('float32')/255
        X_val.append(cv2.cvtColor(im, cv2.COLOR_GRAY2RGB))

    for label in tqdm_notebook(Y_val_pre):
        Y_val.append(label[0])
        
    h5f.close()
    return X_val, Y_val
    
def load_data_batch(start,stop):
    h5f = h5py.File(train_path, 'r')
    X_train_pre = h5f['X'][start:stop]
    Y_train_pre = h5f['Y'][start:stop]

    X_train = []
    Y_train = []

    for image in tqdm_notebook(X_train_pre):
        im = image*255
        eq = cv2.equalizeHist(im.astype('uint8'))
        im = eq.astype('float32')/255
        X_train.append(cv2.cvtColor(im, cv2.COLOR_GRAY2RGB))

    for label in tqdm_notebook(Y_train_pre):
        Y_train.append(label[0])
        
    h5f.close()        
    return X_train, Y_train


In [5]:
import os
if 'X_val.npy' in os.listdir('./'):
    print('loading from file...')
    X_val = np.load('X_val.npy')
    Y_val = np.load('Y_val.npy')
    X_train = np.load('X_train.npy')
    Y_train = np.load('Y_train.npy')
    print('Data loaded')
else:
    X_val, Y_val = load_val()
    X_train, Y_train = load_data_batch(0,10000)
    np.save('X_val',X_val)
    np.save('Y_val',Y_val)
    np.save('X_train',X_train)
    np.save('Y_train',Y_train)


loading from file...
Data loaded


In [28]:
BW_train = []
BW_val = []

for image in tqdm_notebook(X_train):
    #BW_train.append(cv2.cvtColor(image, cv2.COLOR_RGB2GRAY))
    BW_train.append(cv2.resize(image, (IMG_SIZE,IMG_SIZE)))
    
for image in tqdm_notebook(X_val):
    #BW_val.append(cv2.cvtColor(image, cv2.COLOR_RGB2GRAY))
    BW_val.append(cv2.resize(image, (IMG_SIZE,IMG_SIZE)))
X_train = BW_train
X_val = BW_val

In [29]:
X_train = np.array(X_train)
X_val = np.array(X_val)

In [31]:
print(X_train.shape, X_val.shape, Y_val.shape, Y_train.shape)

(10000, 128, 128) (2000, 128, 128) (2000, 2) (10000, 2)


In [9]:
Y_train = 1 - Y_train.astype('int')
Y_train = np.eye(2)[Y_train]

Y_val = 1 - Y_val.astype('int')
Y_val = np.eye(2)[Y_val]

In [51]:
X_train = X_train[:5000].reshape(-1,IMG_SIZE,IMG_SIZE,1)
Y_train = Y_train[:5000]

X_val = X_val[:500].reshape(-1,IMG_SIZE,IMG_SIZE,1)
Y_val = Y_val[:500]


In [52]:
from keras import models
from keras import layers
from keras import optimizers
from keras import callbacks
from keras import backend as K

# Import the network architecture
IMG_WIDTH = IMG_SIZE
IMG_HEIGHT = IMG_SIZE
IMG_DEPTH = 1
BATCH_SIZE = 64
NB_EPOCHS = 15

shape, classes = (IMG_WIDTH, IMG_HEIGHT, IMG_DEPTH), 2

In [60]:
import numpy as np
from keras import layers, models, optimizers
from keras import backend as K
from keras.utils import to_categorical
from capsulelayers import CapsuleLayer, PrimaryCap, Length, Mask

K.set_image_data_format('channels_last')


def CapsNet(input_shape, n_class, num_routing):
    """
    A Capsule Network
    :param input_shape: data shape, 3d, [width, height, channels]
    :param n_class: number of classes
    :param num_routing: number of routing iterations
    :return: Two Keras Models, the first one used for training, and the second one for evaluation.
            `eval_model` can also be used for training.
    """
    x = layers.Input(shape=input_shape)

    # Layer 1: Just a conventional Conv2D layer
    conv1 = layers.Conv2D(filters=64, kernel_size=9, strides=1, padding='valid', activation='relu', name='conv1')(x)

    # Layer 2: Conv2D layer with `squash` activation, then reshape to [None, num_capsule, dim_capsule]
    primarycaps = PrimaryCap(conv1, dim_capsule=8, n_channels=32, kernel_size=9, strides=2, padding='valid')

    # Layer 3: Capsule layer. Routing algorithm works here.
    digitcaps = CapsuleLayer(num_capsule=n_class, dim_capsule=16, num_routing=num_routing,
                             name='digitcaps')(primarycaps)

    # Layer 4: This is an auxiliary layer to replace each capsule with its length. Just to match the true label's shape.
    # If using tensorflow, this will not be necessary. :)
    out_caps = Length(name='capsnet')(digitcaps)

    # Decoder network.
    y = layers.Input(shape=(n_class,))
    masked_by_y = Mask()([digitcaps, y])  # The true label is used to mask the output of capsule layer. For training
    masked = Mask()(digitcaps)  # Mask using the capsule with maximal length. For prediction

    # Shared Decoder model in training and prediction
    decoder = models.Sequential(name='decoder')
    decoder.add(layers.Dense(128, activation='relu', input_dim=16*n_class))
    decoder.add(layers.Dense(256, activation='relu'))
    decoder.add(layers.Dense(np.prod(input_shape), activation='sigmoid'))
    decoder.add(layers.Reshape(target_shape=input_shape, name='out_recon'))

    # Models for training and evaluation (prediction)
    train_model = models.Model([x, y], [out_caps, decoder(masked_by_y)])
    #eval_model = models.Model(x, [out_caps, decoder(masked)])
    return train_model#, eval_model


def margin_loss(y_true, y_pred):
    """
    Margin loss for Eq.(4). When y_true[i, :] contains not just one `1`, this loss should work too. Not test it.
    :param y_true: [None, n_classes]
    :param y_pred: [None, num_capsule]
    :return: a scalar loss value.
    """
    L = y_true * K.square(K.maximum(0., 0.9 - y_pred)) + 0.5 * (1 - y_true) * K.square(K.maximum(0., y_pred - 0.1))

    return K.mean(K.sum(L, 1))


In [61]:
model = CapsNet([IMG_SIZE,IMG_SIZE,1], 2, 3)

model.compile(
    loss='binary_crossentropy',
    optimizer=optimizers.Adam(lr=2e-6),
    metrics=['acc'])

reduce_learning = callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=2,
    verbose=1,
    mode='auto',
    epsilon=0.0001,
    cooldown=2,
    min_lr=0)

eary_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=7,
    verbose=1,
    mode='auto')

cb = [TQDMNotebookCallback()] #,eary_stopping, reduce_learning]

In [62]:
# Train the the model
history = model.fit(
    [X_train, Y_train], [Y_train, X_train],
    epochs=NB_EPOCHS,
    batch_size = 16,
    validation_data=[[X_val, Y_val], [Y_val, X_val]],
    verbose = 2,
    callbacks=cb
)

Train on 5000 samples, validate on 500 samples


Epoch 1/15


ResourceExhaustedError: OOM when allocating tensor of shape [2,100352,16,8] and type float
	 [[Node: training_9/Adam/zeros_4 = Const[dtype=DT_FLOAT, value=Tensor<type: float shape: [2,100352,16,8] values: [[[0 0 0]]]...>, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]

Caused by op 'training_9/Adam/zeros_4', defined at:
  File "D:\anaconda\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "D:\anaconda\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "D:\anaconda\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "D:\anaconda\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "D:\anaconda\lib\site-packages\ipykernel\kernelapp.py", line 486, in start
    self.io_loop.start()
  File "D:\anaconda\lib\site-packages\zmq\eventloop\ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File "D:\anaconda\lib\site-pa

In [63]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.title('Training and validation accuracy')
plt.plot(epochs, acc, 'red', label='Training acc')
plt.plot(epochs, val_acc, 'blue', label='Validation acc')
plt.legend()

plt.figure()
plt.title('Training and validation loss')
plt.plot(epochs, loss, 'red', label='Training loss')
plt.plot(epochs, val_loss, 'blue', label='Validation loss')

plt.legend()

plt.show()

NameError: name 'history' is not defined

In [64]:
max(val_acc)

NameError: name 'val_acc' is not defined

### [0, 1] == Normal == 0

### [1, 0] == Abnormal == 1

### [Abnormalness, Normalness]